# Выбор модели для последующего трансферного обучения в TensorFlow

## Задача

Выбрать оптимальную модель, предварительно обученную на наборе данных *ImageNet*: *VGG16*, *VGG19*, *ResNet50V1*, *ResNet50V2*, *ResNet101V1*, *ResNet101V2*, *ResNet152V1*, *ResNet152V2*.

## Данные

40 000 цветных картинок бетона размером 227x227 пикселей, 20 000 из которых с стрещиной, другие 20 000 целый  
Данные взяты из курса [AI Capstone Project with Deep Learning](https://www.coursera.org/learn/ai-deep-learning-capstone?specialization=ai-engineer), явлюющийся заключительным курсом [IBM AI Engineering Professional Certificate](https://www.coursera.org/professional-certificates/ai-engineer) на сайте [coursera.org](https://www.coursera.org/)  
[Данные](https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip)  

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip

--2023-04-28 16:00:47--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/images/concrete_crack_images_for_classification.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 245259777 (234M) [application/zip]
Saving to: ‘concrete_crack_images_for_classification.zip’

concrete_crack_imag 100%[===================>] 233.90M  34.5MB/s    in 7.4s    

2023-04-28 16:00:56 (31.6 MB/s) - ‘concrete_crack_images_for_classification.zip’ saved [245259777/245259777]



In [2]:
!mkdir ./data
!mkdir ./data/concrete
!mkdir ./models

In [ ]:
!unzip concrete_crack_images_for_classification.zip -d ./data/concrete

## Расчёты

In [4]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications import vgg16, vgg19, resnet50, resnet, resnet_v2
from tensorflow.keras.layers import Dense

In [5]:
def get_model(name_model="VGG16"):
    
    # входной слой модели
    i = tf.keras.layers.Input([None, None, 3], dtype = tf.uint8)
    x = tf.cast(i, tf.float32)

    # инициализация модели, с включённым преобразование данных, и её весов
    if name_model == "VGG16":

        x = vgg16.preprocess_input(x)
        x = vgg16.VGG16(include_top=False,
                        weights="imagenet",
                        pooling="avg")(x)
        
    
    elif name_model == "VGG19":

        x = vgg19.preprocess_input(x)
        x = vgg19.VGG19(include_top=False,
                        weights="imagenet",
                        pooling="avg")(x)
    
    elif name_model == "ResNet50V1":

        x = resnet50.preprocess_input(x)
        x = resnet50.ResNet50(include_top=False,
                              weights="imagenet",
                              pooling="avg")(x)

    elif name_model == "ResNet50V2":

        x = resnet_v2.preprocess_input(x)
        x = resnet_v2.ResNet50V2(include_top=False,
                                 weights="imagenet",
                                 pooling="avg")(x)

    elif name_model == "ResNet101V1":
        
        x = resnet.preprocess_input(x)
        x = resnet.ResNet101(include_top=False,
                             weights="imagenet",
                             pooling="avg")(x)

    elif name_model == "ResNet101V2":
        
        x = resnet_v2.preprocess_input(x)
        x = resnet_v2.ResNet101V2(include_top=False,
                                  weights="imagenet",
                                  pooling="avg")(x)

    elif name_model == "ResNet152V1":
        
        x = resnet.preprocess_input(x)
        x = resnet.ResNet152(include_top=False,
                             weights="imagenet",
                             pooling="avg")(x)

    elif name_model == "ResNet152V2":
        
        x = resnet_v2.preprocess_input(x)
        x = resnet_v2.ResNet152V2(include_top=False,
                                  weights="imagenet",
                                  pooling="avg")(x)
    
    # добавление полносвязного слоя
    x = Dense(units=2, activation="softmax")(x)

    # создание модели
    model = tf.keras.Model(inputs=i, outputs=x, name=name_model)

    # заморозка весов свёрточной модели
    model.layers[-2].trainable = False

    return model

In [6]:
# загрузка пакетированных данных для теста и валидации
train_data, val_data = image_dataset_from_directory(directory="./data/concrete",
                                                    label_mode="categorical",
                                                    batch_size=100,
                                                    image_size=(224,224),
                                                    shuffle=True,
                                                    seed=0,
                                                    validation_split=0.25,
                                                    subset="both",
                                                    interpolation="bilinear")

Found 40000 files belonging to 2 classes.
Using 30000 files for training.
Using 10000 files for validation.


In [7]:
all_names = ["VGG16", "VGG19", "ResNet50V1", "ResNet50V2", "ResNet101V1", "ResNet101V2", "ResNet152V1", "ResNet152V2"]

for name in all_names:

    print(f"\nModel : {name}\n")

    # создание модели
    model = get_model(name)

    # установление оптимизатора, функции ошибки и метрики
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss="categorical_crossentropy",
                  metrics=['accuracy'])
    
    print(f"\nЧисло параметров сети: {model.count_params() / 1e6:.3f} * 10^6")

    # параметры в формате float32, т.е. 1 параметр занимает в памяти 32 бита / 4 байта
    print(f"Занимают в памяти: {model.count_params() * 4 / (1024 ** 2):.3f} Мб\n")

    # обучение модели
    model.fit(train_data, epochs=3, validation_data=val_data)
    print("\n"+"="*40)


Model : VGG16

58889256/58889256 [==============================] - 2s 0us/step

Число параметров сети: 14.716 * 10^6
Занимают в памяти: 56.136 Мб

Epoch 1/3
300/300 [==============================] - 188s 559ms/step - loss: 0.0701 - accuracy: 0.9786 - val_loss: 0.0232 - val_accuracy: 0.9938
Epoch 2/3
300/300 [==============================] - 216s 720ms/step - loss: 0.0163 - accuracy: 0.9964 - val_loss: 0.0148 - val_accuracy: 0.9954
Epoch 3/3
300/300 [==============================] - 174s 579ms/step - loss: 0.0111 - accuracy: 0.9973 - val_loss: 0.0112 - val_accuracy: 0.9965


Model : VGG19

80134624/80134624 [==============================] - 3s 0us/step

Число параметров сети: 20.025 * 10^6
Занимают в памяти: 76.391 Мб

Epoch 1/3
300/300 [==============================] - 247s 818ms/step - loss: 0.1214 - accuracy: 0.9590 - val_loss: 0.0223 - val_accuracy: 0.9959
Epoch 2/3
300/300 [==============================] - 220s 731ms/step - loss: 0.0166 - accuracy: 0.9965 - val_loss: 0.0126

## Результаты

Как видно, все модели, по истечению трёх эпох обучения, достигли точности свыше 99%. (Именно для этих данных)  
Так как помимо полносвязного слоя в конце, можно ещё слегка поднастроить веса свёрточных слоёв, то обращать внимание на сотые доли процента не имеет значения.  
Время обучения так же не является наиболее важным критерием для моделей, так как модель обучается единожды и потом это не требуется, к тому же все модели обучаются примерно за одно и тоже время.  
А вот количество параметров, а следовательно занимаемый объём памяти моделью, на мой взгляд более важный аспект.  
Из-за всего вышеперечисленного будем использовать самую легковесную модель - _**VGG16**_ с объёмом в ~ 56 Мб